In [43]:
from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers sentence-transformers
!pip install scrapy pandas transformers sentence-transformers scikit-learn
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.0 MB/s eta 0:00:00


In [18]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [68]:
import pandas as pd

data = {
    "job_id": [1, 2],
    "skills": ["Python; Java; SQL", "Python;TensorFlow;PyTorch"]
}
job_data = pd.DataFrame(data)
print(job_data.columns)
def preprocess_skills(skills):
    if pd.isnull(skills):
        return ""

    return skills.lower().replace(';', ', ').strip()


job_data['cleaned_skills'] = job_data['skills'].apply(preprocess_skills)

print(job_data)


Index(['job_id', 'skills'], dtype='object')
   job_id                     skills               cleaned_skills
0       1          Python; Java; SQL          python,  java,  sql
1       2  Python;TensorFlow;PyTorch  python, tensorflow, pytorch


In [76]:
import pandas as pd
import re

job_data = pd.read_csv('/content/drive/MyDrive/JobGrind.io/job_grind/job_data (1).csv', encoding='ISO-8859-1')
print(job_data.columns)
print(job_data.head())


def preprocess_text(text):
    if pd.isnull(text):
        return ""

    return re.sub(r'[^a-zA-Z\s]', '', text).lower()

job_data['cleaned_desription'] = job_data['job_title'].apply(preprocess_text)

def preprocess_skills(skills):
    if pd.isnull(skills):
        return ""
    return skills.lower().replace(';', ', ')

job_data['cleaned_skills'] = job_data['skills'].apply(preprocess_skills)

job_data['cleaned_location'] = job_data['location'].fillna('Unknown')

print(job_data[['cleaned_description', 'cleaned_skills', 'cleaned_location']].head())

cleaned_file_path = '/content/drive/MyDrive/JobGrind.io/job_grind/cleanjob_grind.csv'
job_data.to_csv(cleaned_file_path, index=False)
print(f"Cleaned dataset saved to {cleaned_file_path}")


Index(['job_id', 'skills', 'cleaned_description', 'cleaned_skills',
       'cleaned_location', 'job_type', 'location', 'job_title', 'job_url'],
      dtype='object')
   job_id                     skills  cleaned_description  cleaned_skills  \
0       1            Python;Java;SQL                  NaN             NaN   
1       2  Python;R;Machine Learning                  NaN             NaN   
2       3  JavaScript;HTML;CSS;React                  NaN             NaN   
3       4      Node.js;React;MongoDB                  NaN             NaN   
4       5      Figma;Sketch;Adobe XD                  NaN             NaN   

   cleaned_location   job_type              location  \
0               NaN  Full-time  Bengaluru, Karnataka   
1               NaN  Permanent  Bengaluru, Karnataka   
2               NaN  Part-time  Bengaluru, Karnataka   
3               NaN  Full-time  Bengaluru, Karnataka   
4               NaN  Full-time  Bengaluru, Karnataka   

                                  

In [77]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

job_data['cleaned_description'] = job_data['cleaned_description'].astype(str)

job_embeddings = model.encode(job_data['cleaned_description'].tolist())

print(job_embeddings)

np.save('/content/drive/MyDrive/JobGrind.io/job_grind/job_embeddings.npy', job_embeddings)
print("Job embeddings generated and saved!")

[[-0.46067664  0.5258659  -0.02412613 ...  0.26479387 -0.32959247
  -0.17524558]
 [-0.46067664  0.5258659  -0.02412613 ...  0.26479387 -0.32959247
  -0.17524558]
 [-0.46067664  0.5258659  -0.02412613 ...  0.26479387 -0.32959247
  -0.17524558]
 ...
 [-0.46067664  0.5258659  -0.02412613 ...  0.26479387 -0.32959247
  -0.17524558]
 [-0.46067664  0.5258659  -0.02412613 ...  0.26479387 -0.32959247
  -0.17524558]
 [-0.46067664  0.5258659  -0.02412613 ...  0.26479387 -0.32959247
  -0.17524558]]
Job embeddings generated and saved!


In [78]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

user_skills = input()

def preprocess_text(text):
    import re
    return re.sub(r'[^a-zA-Z\s]', '', str(text)).lower()

user_skills_cleaned = preprocess_text(user_skills)

user_embedding = model.encode([user_skills_cleaned])

print(user_skills_cleaned)

print("User embedding generated!")



python sql java
python sql java
User embedding generated!


In [84]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

job_embeddings = normalize(job_embeddings, axis=1)
user_embedding = normalize(user_embedding.reshape(1, -1), axis=1)

# Weighted feature similarity
title_weight = 0.5
description_weight = 0.4
location_weight = 0.1

title_similarity = cosine_similarity(user_title_embedding, job_title_embeddings)
description_similarity = cosine_similarity(user_description_embedding, job_description_embeddings)
location_similarity = cosine_similarity(user_location_embedding, job_location_embeddings)

# Combine weighted similarities
combined_similarity = (
    title_weight * title_similarity +
    description_weight * description_similarity +
    location_weight * location_similarity
)

# Add combined similarity scores to job data
job_data['similarity_score'] = combined_similarity[0]

# Sort and display recommendations
recommended_jobs = job_data.sort_values(by='similarity_score', ascending=False)
print(recommended_jobs[['job_title', 'similarity_score', 'job_url']].head())



                                         job_title  similarity_score  \
0     Sr. Fullstack Developer ( Reactjs & Nodejs )          0.084135   
1                .Net Full Stack Developer- Junior          0.084135   
122                           Lead DevOps Engineer          0.084135   
123  DevOps Engineer(8-12yrs) | Network & Security          0.084135   
124                     Data Engineer (AWS DevOps)          0.084135   

                                               job_url  
0    https://in.indeed.com/viewjob?jk=3bba0c972f7fc...  
1    https://in.indeed.com/viewjob?jk=ff67c84f1a81f...  
122  https://in.indeed.com/viewjob?jk=56a78e6aa5ea4...  
123  https://in.indeed.com/viewjob?jk=de3d9f69ed9c0...  
124  https://in.indeed.com/viewjob?jk=bad48be29b2d4...  
User embedding shape: (1, 384)
Job embeddings shape: (192, 384)
Sample job embedding: [-5.62421940e-02  6.42008930e-02 -2.94546434e-03 -5.04706539e-02
 -3.36342230e-02 -6.09433576e-02  3.97253558e-02  3.37631255e-02
  5.15727

In [81]:
output_path = '/content/drive/MyDrive/JobGrind.io/job_grind/reccommended_jobs.csv'
reccommended_jobs[['job_title', 'similarity_score', 'location','job_url']].to_csv(output_path, index=False)
print(f"Recommended jobs saved to: {output_path}")

Recommended jobs saved to: /content/drive/MyDrive/JobGrind.io/job_grind/reccommended_jobs.csv


In [ ]:
!curl https://quizapi.io/OO0VyP5dukH7XJuruNxKZDc7iBpIcXJH6qGBt44k/v1/questions

<!DOCTYPE html>
<html lang="en">
<head>

    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

      <title>Quiz API - Contact us</title>
  <meta name="Description" content="Test your knowledge or easily embed a quiz on your website with the Quiz API">
  <meta name="keywords" content="quiz, quizapi, test, bash quiz, linux quiz, php quiz, tech quiz, interview questions, html quiz, docker quiz, kubernetes quiz">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <meta property="og:image" content="https://quizapi.io/storage/QuziApi_Logo_Dark.png" />
  <meta property="og:title" content="Quiz API - Contact us" />
  <meta property="og:description" content="Test your knowledge or easily embed a quiz on your website with the Quiz API" />	
  <meta name="twitter:card" content="summary">
  <meta name="twitter:site" content="quizapi.io" />
  <meta name="twitter:title" content="Quiz API - Contact us" />
  <meta n

In [82]:
import requests
import random


API_KEY = "OO0VyP5dukH7XJuruNxKZDc7iBpIcXJH6qGBt44k"

category=["DevOps","Cloud","Networking","Programming","Linux"]

def fetch_questions(limit=5, category="DevOps"):
    url = "https://quizapi.io/api/v1/questions"
    headers = {"X-Api-Key": API_KEY}
    params = {"limit": limit}

    if category:
        params["category"] = category

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch questions. Check your API key or parameters.")
        return []

def play_quiz(questions):
    score = 0

    for idx, question in enumerate(questions, 1):
        print(f"\nQuestion {idx}: {question['question']}")

        options = []
        for key, value in question['answers'].items():
            if value:
                options.append(value)

        random.shuffle(options)
        for i, option in enumerate(options):
            print(f"{i + 1}. {option}")

        try:
            answer = int(input("Enter the number of your choice: "))
            correct_answers = question['correct_answers']

            if options[answer - 1] in [k for k, v in correct_answers.items() if v == "true"]:
                print("Correct!")
                score += 1
            else:
                print("Wrong!")
        except (ValueError, IndexError):
            print("Invalid choice!")

    print(f"\nQuiz Over! Your score: {score}/{len(questions)}")

if __name__ == "__main__":
    print("Welcome to the Quiz Generator!")
    questions = fetch_questions(limit=10)
    if questions:
        play_quiz(questions)


Welcome to the Quiz Generator!

Question 1: In OpenShift, which resource is responsible for granting external access to an application via HTTP or HTTPS?
1. NodePort
2. Route
3. Service
4. Ingress
Enter the number of your choice: 2
Wrong!

Question 2: Which command would you use to get the hostname of a remote host with Ansible
1. ansible your_server -m setup -a field=*hostname*
2. ansible your_server -m env -a filter=*hostname*
3. ansible your_server -m setup -a get=*hostname*
4. ansible your_server -m setup -a filter=*hostname*
Enter the number of your choice: 1
Wrong!

Question 3: How do you manage permissions in OpenShift?
1. By setting firewall rules
2. Through external LDAP configurations
3. Only via Kubernetes pods
4. Using the OpenShift RBAC (Role-Based Access Control)
Enter the number of your choice: 3
Wrong!

Question 4: In an OpenShift environment, which etcd command would you use to verify the integrity of a snapshot file?
1. etcdctl snapshot restore <snapshot-file>
2. etcd